In [1]:
import numpy as np

In [2]:
ano_i = 1900.00
ano_f = 2000.00

lat_i = -89.00
lat_f = 89.00

lon_i = -180.00
lon_f = 180.00

d_ano = 10.0
d_coord = 2.0

In [3]:
# ok

lats = np.arange(lat_i, lat_f+d_coord, d_coord)
lats = lats.reshape(len(lats),1)
lons = np.arange(lon_i, lon_f+d_coord, d_coord)
lons = lons.reshape(len(lons),1)
anos = np.arange(ano_i, ano_f+d_ano, d_ano)
anos = anos.reshape(len(anos),1)

In [4]:
#coluna de longitudes ok

lons_2 = np.repeat(lons, len(lats), axis=1).reshape(len(lons)*len(lats), 1)
lons_3 = np.repeat(lons_2, len(anos), axis=1)
lons_3 = lons_3.T.reshape(lons_3.size, 1)

In [5]:
# coluna de latitudes ok
lats_2 = np.repeat(lats, len(lons), axis=1).T.reshape(len(lons)*len(lats), 1)
lats_3 = np.repeat(lats_2, len(anos), axis=1)
lats_3 = lats_3.T.reshape(lats_3.size, 1)

In [6]:
# coluna de anos ok
anos_2 = np.repeat(anos, len(lons_2), axis=1).reshape(len(lons)*len(lats)*len(anos), 1)

In [7]:
M = np.concatenate((lats_3, lons_3, anos_2), axis=1)
M.shape

(179190, 3)